In [2]:
# Use autoplot 207's code for this app
import sys
from datetime import datetime

import numpy as np
from pyproj import Transformer

import geopandas as gpd
import pandas as pd
from iemweb.autoplot.scripts200.p207 import USEME, add_zeros, compute_grid_bounds, do_analysis, load_data
from matplotlib.patches import Rectangle
from pyiem.nws.vtec import NWS_COLORS
from pyiem.plot import MapPlot, nwssnow
from pyiem.database import get_sqlalchemy_conn
from pyiem.reference import Z_OVERLAY
from pyiem.util import utc
from shapely.geometry import Point

In [3]:
STORM_NUMBER = 9
REQUIRES_DAYS = 1
WINTER = "2025-2026"
TITLE = "28-29 December 2025"
SUBTITLE = "8 AM 29 December 2025"
SETPOINT_LOCS = {}
# naive US Central local time
sts = datetime(2025, 12, 28, 16)
ets = datetime(2025, 12, 29, 18)
# Get available data
ctx = {
    "coop": "yes",
    "cocorahs": "yes",
    "t": "state",
    "sz": 30,
    "z": "yes",
    "f": "linear",
    "v": "snow",
    "wfo": "DMX",
}
# figure out our grid bounds
ctx["bnds2163"] = compute_grid_bounds(ctx, "IA")
df = load_data(ctx, sts, ets)
df = df[~df["nwsli"].isin(["DSXI4", "DMX"])]
# add zeros and QC
df = add_zeros(df, ctx)
df.to_csv("raw.csv")

In [4]:
def overlay_ice(mp):
    """Add plotted ice storm."""
    with get_sqlalchemy_conn("postgis") as conn:
        df = pd.read_sql(
            """
        SELECT st_x(geom) as lon, st_y(geom) as lat, magnitude from lsrs WHERE
        typetext in ('ICE STORM', 'FREEZING RAIN') and magnitude > 0
        and valid > %s and valid < %s and state = 'IA'
        """,
            conn,
            params=(sts - datetime.timedelta(days=1), ets),
        )
    print(df[df["state"] == "IA"])
    mp.plot_values(
        df.lon.values,
        df.lat.values,
        df.magnitude.values,
        fmt="%.2f",
        labelbuffer=1,
        color="purple",
    )


def workflow(ctx, df, isfinal=False, lower=0, upper=2):
    # do gridding
    df2 = df[df[USEME]]
    lons, lats, vals = do_analysis(df2, ctx)
    mp = MapPlot(
        sector="state",
        state=ctx["csector"],
        axisbg="white",
        title="%s - IEM Snowfall Total Analysis" % (TITLE,),
        subtitle=(
            f"Snowfall totals till {SUBTITLE} from NWS COOP, LSR, CoCoRaHS Reports; "
            f"IEM {WINTER} Winter Storm #{STORM_NUMBER}"
        ),
        twitter=True,
    )
    cmap = nwssnow()
    # cmap = get_cmap("Greens")
    ramp = [0.1, 1, 2, 3, 4, 6, 8, 12, 18, 24, 30, 36]
    # ramp = [0.1, 1, 2, 3, 4]
    mp.contourf(lons, lats, vals, np.array(ramp), cmap=cmap, clip_on=True)
    df_useme_plot = df2[(df2["val"] >= lower) & (df2["val"] < upper)]
    print(df[df["state"] == "IA"])
    mp.drawcounties()
    # overlay_ice(mp)
    if isfinal:
        mp.drawcities()
    else:
        mp.plot_values(
            df_useme_plot["lon"],
            df_useme_plot["lat"],
            df_useme_plot["val"].values,
            "%s",
            labels=df_useme_plot["nwsli"].values,
            textsize=10,
            labeltextsize=10,
            labelbuffer=1,
        )
    return mp

In [21]:
def add_setpoints(setpoints):
    """Manual things."""
    for sp, val in setpoints:
        df.at[10000 + sp, "geo"] = Point(
            SETPOINT_LOCS[sp][0], SETPOINT_LOCS[sp][1]
        )
        df.at[10000 + sp, "val"] = val
        df.at[10000 + sp, USEME] = True
        df.at[10000 + sp, "plotme"] = True


def draw_setpoints(mp):
    """Add some points where manual obs could be inserted."""
    xlim = mp.panels[0].ax.get_xlim()
    ylim = mp.panels[0].ax.set_ylim()
    sz = ctx["sz"] * 1000.0
    i = 0
    trans = Transformer.from_proj(mp.panels[0].crs, 2163, always_xy=True)
    for y in np.arange(ylim[0] + sz / 2, ylim[1], sz):
        for x in np.arange(xlim[0] + sz / 2, xlim[1], sz):
            mp.panels[0].ax.text(x, y, f"{i}", ha="center", va="center")
            # Need to store the x, y in 2163, which is what p207 uses :/
            (xx, yy) = trans.transform(x, y)
            SETPOINT_LOCS[i] = [xx, yy]
            i += 1

def plot_warnings(mp):
    with get_sqlalchemy_conn("postgis") as conn:
        gdf = gpd.read_postgis(
            """
            SELECT simple_geom from warnings w JOIN ugcs u on (w.gid = u.gid) WHERE w.phenomena = 'BZ'
            and w.issue > '2025-12-28 00:00' and w.issue < '2025-12-30 00:00' and substr(w.ugc, 1, 2) = 'IA'
            and vtec_year = 2025
            """,
            conn,
            geom_col="simple_geom",
        )
    gdf.to_crs(mp.panels[0].crs).plot(
        ax=mp.panels[0].ax,
        aspect=None,
        edgecolor=NWS_COLORS["BZ.W"],
        facecolor="None",
        zorder=Z_OVERLAY,
        linewidth=2,
    )
    p0 = Rectangle((0, 0), 1, 1, ec=NWS_COLORS["BZ.W"], fc="None")
    p1 = Rectangle((0, 0), 1, 1, ec=NWS_COLORS["SQ.W"], fc="None")
    mp.panels[0].ax.legend((p0, p1), ("Blizzard Warning", "Snow Squall Warning"), loc=1).set_zorder(
        1000
    )

def plotsqw(mp):
    with get_sqlalchemy_conn("postgis") as conn:
        gdf = gpd.read_postgis(
            f"SELECT geom from sbw_{sts.year} w WHERE w.phenomena = 'SQ' and w.issue > %s and w.issue < %s",
            conn,
            params=(datetime(2025, 12,28), ets),
        )
    gdf.to_crs(mp.panels[0].crs).plot(
        ax=mp.panels[0].ax,
        aspect=None,
        edgecolor='k',
        facecolor="None",
        zorder=Z_OVERLAY,
        linewidth=4,
    )
    gdf.to_crs(mp.panels[0].crs).plot(
        ax=mp.panels[0].ax,
        aspect=None,
        edgecolor=NWS_COLORS["SQ.W"],
        facecolor="None",
        zorder=Z_OVERLAY,
        linewidth=2,
    )
    #p0 = Rectangle((0, 0), 1, 1, ec=NWS_COLORS["SQ.W"], fc="None")
    #mp.panels[0].ax.legend((p0,), ("Snow Squall Warning",), loc=1).set_zorder(
    #    1000
    #)

In [6]:
# Optional filter for requiring_days
if REQUIRES_DAYS > 1:
    df = df[(df["source"] == "LSR") | (df["count"] >= REQUIRES_DAYS)]
print(df[df["wfo"] == "DMX"]["source"].value_counts())

source
COCORAHS    15
COOP        11
Name: count, dtype: int64


In [22]:
def main():
    setpoints = [
    ]
    if setpoints:
        add_setpoints(setpoints)
    cull = [
        'IA-MR-6',
        'IA-SH-8',
        'MPTI4',
        'LSXI4',
        'IA-FM-6',
    ]
    if cull:
        df.loc[df["nwsli"].isin(cull), USEME] = False
    hardcode = [
        ('IA-PK-115', 1),
        ('Z6', 0.5),
        ('DLDI4', 1),
        ('IA-SR-58', 1),
        ('IA-PA-5', 1),
    ]
    for nwsli, val in hardcode:
        df.loc[df["nwsli"] == nwsli, "val"] = val

    ctx["csector"] = "IA"
    mp = workflow(ctx, df, isfinal=True, lower=0, upper=1)
    #draw_setpoints(mp)

    plot_warnings(mp)
    plotsqw(mp)
    res = mp.postprocess(filename="251229.png")
    mp.close()


main()

    state  wfo     val        lon        lat                             geo  \
137    IA  ARX  3.0000 -91.830000  42.880000  POINT (664437.626 -202476.225)   
170    IA  ARX  1.3000 -90.960000  42.680000    POINT (737255.5 -217160.311)   
174    IA  DVN  1.1000 -91.640000  42.020000  POINT (689350.435 -296162.409)   
175    IA  DVN  1.0000 -91.620000  42.060000  POINT (690550.775 -291564.058)   
183    IA  DVN  1.0000 -90.920000  41.870000  POINT (750222.465 -306449.653)   
221    IA  DMX  3.0000 -94.833300  43.416700  POINT (417001.846 -162837.311)   
222    IA  DMX  3.0000 -92.933300  42.066700  POINT (582630.807 -301019.509)   
225    IA  ARX  2.5000 -91.830250  42.882620  POINT (664388.846 -202188.022)   
228    IA  OAX  2.3000 -96.016700  41.816700  POINT (330068.669 -345917.663)   
230    IA  DVN  2.0000 -90.735000  42.399800  POINT (758952.411 -246112.902)   
233    IA  FSD  2.0000 -95.100000  43.423100  POINT (395467.661 -163455.245)   
238    IA  DVN  1.8000 -91.450000  42.46

## 